In [1]:
import pandas as pd
import openai
import llama_index

import pandas as pd
from dotenv import load_dotenv
import os
from llama_index.core import Document
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.mistralai import MistralAIEmbedding
from sentence_transformers import SentenceTransformer

from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from IPython.display import Markdown, display
import chromadb
from chromadb.config import Settings

/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
from dotenv import load_dotenv
import openai

# Load environment variables from .env file
load_dotenv(dotenv_path='../.env')

# Accessing the API key from environment variables
openai.api_key = os.getenv("OPENAI_API_KEY")

chroma_server_host = os.getenv("CHROMA_SERVER_HOST")
chroma_server_http_port = os.getenv("CHROMA_SERVER_HTTP_PORT")

In [3]:
chroma_client = chromadb.HttpClient(host=chroma_server_host, port=chroma_server_http_port)

2024-04-21 12:28:59,799 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


In [13]:
import PyPDF2

def extract_text_from_pdf(file_path):
    with open(file_path, 'rb') as pdf_file_obj:
        pdf_reader = PyPDF2.PdfReader(pdf_file_obj)
        text = ''
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

# Usage example
text = extract_text_from_pdf('./data/BIR_raw_data.pdf')
documents = [Document(text=text)]


In [12]:
embed_model = OpenAIEmbedding(model="text-embedding-3-small")
db = chromadb.HttpClient(host=chroma_server_host, port=chroma_server_http_port)


2024-04-21 12:34:45,823 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


In [15]:
# save to disk
chroma_collection = db.get_or_create_collection("BIR_collection")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context, embed_model=embed_model
)

2024-04-21 12:35:05,071 ERROR chromadb.telemetry.product.posthog: Failed to send telemetry event ClientStartEvent: You are using a deprecated configuration of Chroma.

If you do not have data you wish to migrate, you only need to change how you construct
your Chroma client. Please see the "New Clients" section of https://docs.trychroma.com/migration.
________________________________________________________________________________________________

If you do have data you wish to migrate, we have a migration tool you can use in order to
migrate your data to the new Chroma architecture.
Please `pip install chroma-migrate` and run `chroma-migrate` to migrate your data and then
change how you construct your Chroma client.

See https://docs.trychroma.com/migration for more information or join our discord at https://discord.gg/8g5FESbj for help!


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-041Bz***************************************hLFf. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}